In [228]:
import numpy as nd
import pandas as pd
data=pd.read_csv("/content/rawdata.xlsx - inputsheet.csv")
data.head()

,date,time,sensor,location,number,activity,position,location.1
0,1/16/2024,10:10:30 AM,0.0,A1,1,placed,inside,A1
1,1/16/2024,10:12:30 AM,1.0,A2,1,picked,inside,A2
2,1/16/2024,10:28:30 AM,1.0,A3,1,picked,Inside,A3
3,1/16/2024,10:38:20 AM,0.0,A4,1,placed,Inside,A4
4,1/16/2024,10:45:20 AM,1.0,A5,1,placed,inside,A5


In [229]:
print(pd.to_datetime(data['time'][56]).time())

01:00:20


In [230]:
data['time'] = [dt.time() for dt in pd.to_datetime(data['time'])]

<ipython-input-230-fe7468deb9d7>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['time'] = [dt.time() for dt in pd.to_datetime(data['time'])]


In [245]:
data=data.sort_values(['date','time'])
data.reset_index(drop=True,inplace=True)
data.head()

,date,time,sensor,location,number,activity,position,location.1
0,1/16/2024,00:02:20,1.0,A17,1,placed,inside,A17
1,1/16/2024,00:02:20,0.0,A25,1,placed,inside,A25
2,1/16/2024,00:08:20,1.0,A19,1,picked,inside,A19
3,1/16/2024,00:10:20,0.0,A20,1,placed,inside,A20
4,1/16/2024,00:15:20,0.0,A21,1,placed,inside,A21


In [254]:
df=pd.DataFrame()
df['date']=data['date'].unique()
df['inside(sec)']=[0,0,0]
df['outside(sec)']=[0,0,0]
df

,date,inside(sec),outside(sec)
0,1/16/2024,0,0
1,1/17/2024,0,0
2,1/18/2024,0,0


In [255]:
data.isnull().sum()

date          0
time          0
sensor        1
location      0
number        0
activity      0
position      0
location.1    0
dtype: int64

In [256]:
data['date'].describe()['count']

175

In [257]:
def seconds(time_component):
    total_seconds = time_component.hour * 3600 + time_component.minute * 60 + time_component.second
    return total_seconds

In [258]:
dat=data['date'][0]
i=data['date'].describe()['count']
count=0
pos_past="INSIDE"
j=0
for date in data['date'].unique():
  dat=date
  t_inside=0
  t_outside=0
  t_past=data['time'][0]
  t_pres=0

  while(date==dat and count<=i-1):

    if data['position'][count].upper()=="INSIDE"==pos_past and count<=i-1:
      t_pres=data['time'][count]
      if count !=0:

        t_inside=seconds(t_pres)-seconds(t_past) +t_inside

      t_past=t_pres

      pos_past=data['position'][count].upper()

    elif data['position'][count].upper()=="OUTSIDE"==pos_past and count<=i-1:

      t_pres=data['time'][count]
      t_outside=seconds(t_pres)-seconds(t_past)+t_outside

      t_past=t_pres
      pos_past=data['position'][count].upper()

    elif data['position'][count].upper()=="INSIDE"!=pos_past and count<=i-1:

      t_pres=data['time'][count]
      t_outside=seconds(t_pres)-seconds(t_past)+t_outside
      t_past=t_pres

      pos_past=data['position'][count].upper()

    elif data['position'][count].upper()=="OUTSIDE"!=pos_past and count<=i-1:

      t_pres=data['time'][count]
      t_inside=seconds(t_pres)-seconds(t_past)+t_inside
      t_past=t_pres

      pos_past=data['position'][count].upper()

    count=count+1
    if count<=i-1:
      date=data['date'][count]

  df['inside(sec)'][j]=t_inside
  df['outside(sec)'][j]=t_outside
  j=j+1







<ipython-input-258-96fdb25931b9>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inside(sec)'][j]=t_inside
<ipython-input-258-96fdb25931b9>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['outside(sec)'][j]=t_outside
<ipython-input-258-96fdb25931b9>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['inside(sec)'][j]=t_inside
<ipython-input-258-96fdb25931b9>:54: SettingWithCopyWarning: 
A value is trying to be set on a cop

In [259]:
df

,date,inside(sec),outside(sec)
0,1/16/2024,43020,0
1,1/17/2024,47940,0
2,1/18/2024,67267,2813


In [260]:
data.head()

,date,time,sensor,location,number,activity,position,location.1
0,1/16/2024,00:02:20,1.0,A17,1,placed,inside,A17
1,1/16/2024,00:02:20,0.0,A25,1,placed,inside,A25
2,1/16/2024,00:08:20,1.0,A19,1,picked,inside,A19
3,1/16/2024,00:10:20,0.0,A20,1,placed,inside,A20
4,1/16/2024,00:15:20,0.0,A21,1,placed,inside,A21


In [262]:
df1=pd.DataFrame()
df1['date']=data['date'].unique()
df1['placed']=0
df1['picked']=0
df1

,date,placed,picked
0,1/16/2024,0,0
1,1/17/2024,0,0
2,1/18/2024,0,0


In [273]:
dat=data['date'][0]
i=data['date'].describe()['count']
count=0

j=0
for date in data['date'].unique():
  dat=date
  placed=0
  picked=0

  while(date==dat and count<=i-1):
    if data['activity'][count]=="picked" and count<=i-1:
      picked=picked+1
    if data['activity'][count]=="placed" and count<=i-1:
      placed=placed+1

    count=count+1
    if count<=i-1:
      date=data['date'][count]


  df1['placed'][j]=placed
  df1['picked'][j]=picked
  j=j+1


<ipython-input-273-59cef0c94c08>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['placed'][j]=placed
<ipython-input-273-59cef0c94c08>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['picked'][j]=picked
<ipython-input-273-59cef0c94c08>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['placed'][j]=placed
<ipython-input-273-59cef0c94c08>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [274]:
df1

,date,placed,picked
0,1/16/2024,40,40
1,1/17/2024,9,10
2,1/18/2024,39,37
